# Análise de Sentimentos do Twitter
### Hugo Gabriel Bezerra da Silva

Essa atividade tem como objetivo treinar um modelo para classificar tweets em positivos e negativos com base no texto desses tweets.

O modelo foi treinado com um conjunto de quase 26 mil tweets classificados da seguinte forma:

* Se texto contém **:)** então o tweet é Positivo
* Se texto contém **):** então o tweet é Negativo

Tendo esse modelo treinado, ele será aplicado em novos tweets. Para compor a base de teste foram coletados mais de 1600 tweets contendo pelo menos uma das seguntes hashtags:

* #mariellefranco 
* #MariellePresente 
* #QuemMatouMarielle

Como podemos ver o assunto de interesse nesse caso é o assassinato da socióloga, feminista, defensora dos direitos humanos e política brasileira **Marielle Franco** que foi executada, junto com o motorista Anderson Pedro Mathias Gomes, em 14 de março de 2018 em um crime político que chocou o Brasil e o Mundo e paira até hoje sem respostas conclusivas.

In [1]:
import nltk
import re
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn import metrics
from sklearn.model_selection import cross_val_predict

### Dados de Treino

Como dito, contamos com aproximadamente 26 mil tweets para treino. Vale lembrar que foram removidas observações NA dos dados além de alguns passos de tratamento nos dados. 

In [2]:
tweets = pd.read_csv("tweets.csv", encoding='utf-8')
tweets.shape

(25911, 4)

In [3]:
# Remove NaN's
tweets.dropna(inplace=True)

# Remove coluna desnecessária
tweets = tweets.drop(columns=['id2'])
tweets.shape

(25905, 3)

### Tratamento dos Dados

A seguir temos um conjunto de funções que é responsável por fornecer os dados de forma mais "limpa" para o treino do modelo.

São removidos *stopwords* (artigos, preposições e etc.), *links*, menções, identificadores de retweet e caracteres especiais.

Além dísso a categoria do sentimento que é dada como 0 ou 1 é transformada para Negativo e Positivo.

In [4]:
def remove_stopwords(text):
    stopwords = set(nltk.corpus.stopwords.words('portuguese'))
    words = [i for i in text.split() if not i in stopwords]
    return (" ".join(words))

def remove_links(text):
    return re.sub(r"http\S+", "", text)

def remove_mentions(text):
    return re.sub(r"@\w+", "", text)

def remove_retweets(text):
    return re.sub(r"rt\s", "", text)

def remove_special_chars(text):
    text = re.sub(r'[^\w\s]', ' ', text)
    text = re.sub(r"$\d+\W+|\b\d+\b|\W+\d+$", "", text)
    text_with_no_special_chars = re.sub("\s+", " ", text)
    return text_with_no_special_chars

def standardize_text(text):
    text = text.lower()
    text = remove_links(text)
    text = remove_mentions(text)
    text = remove_retweets(text)
    text = remove_stopwords(text)
    text = remove_special_chars(text)
    return text
def sentiment_to_name(n):
    if n == 0 or n == "0":
        n = 'Negativo'
    elif n == 1 or n == "1":
        n = 'Positivo'
    return n

Os dados sem a padronização são:

In [5]:
tweets.head()

,id,text,sentiment
0,7895,Hoje tem vídeo abrindo 65 emp :),1.0
1,7965,Passar a tarde a estudar olha q bonito :),1.0
2,3610,"Tava uma gaja no torneio, ai que filha da puta...",1.0
3,1139,Chateadissima com a eliminação da Red Canids :...,0.0
4,7386,Tô pensando em fazer uma colinha só que a prof...,0.0


Já com a padronização os dados são:

In [6]:
tweets.text = tweets.text.apply(standardize_text)
tweets.sentiment = tweets.sentiment.apply(sentiment_to_name)
tweets.head()

,id,text,sentiment
0,7895,hoje vídeo abrindo emp,Positivo
1,7965,passar tarde estudar olha q bonito,Positivo
2,3610,tava gaja torneio ai filha puta ainda tentei a...,Positivo
3,1139,chateadissima eliminação red canids diria sent...,Negativo
4,7386,tô pensando fazer colinha professora é esperta,Negativo


### Modelo

Para construir o modelo estamos considerando as palavras isoladas e também as bigramas. Usamos também o Bag of Words na nossa representação, nesse tipo de representação não estamos preocupados com a ordem que os termos (palavras) aparecem no documento (no caso tweet), mas sim se um tweets contém uma certa palavra e em que quantidade.

Utilizaremos uma **Rede Bayesiana** visto que esse tipo de classificador costuma ter um bom desempenho na classificação de texto. 

In [7]:
tweets_text = tweets["text"].values
classes = tweets['sentiment'].values

In [8]:
vectorizer = CountVectorizer(ngram_range = (1, 2))
freq_tweets = vectorizer.fit_transform(tweets_text)

modelo = MultinomialNB()
modelo.fit(freq_tweets, classes)

MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True)

### Resultados do Modelo - Treino

Veremos o desempenho do modelo sobre os dados de treino, segundo as métricas:

* Accuracy
* Precision
* Recall
* F1-Score

In [9]:
resultados = cross_val_predict(modelo, freq_tweets, classes, cv = 10)

In [10]:
print("A acurácia do modelo é %.2f nos dados de treino." % metrics.accuracy_score(classes, resultados))

A acurácia do modelo é 0.77 nos dados de treino.


In [11]:
sentimentos = ["Positivo", "Negativo"]
print("O quadro abaixo mostra os valores de precision, recall e f1-score para os dados de treino\n")
print(metrics.classification_report(classes, resultados, sentimentos))

O quadro abaixo mostra os valores de precision, recall e f1-score para os dados de treino

             precision    recall  f1-score   support

   Positivo       0.69      0.57      0.62      8703
   Negativo       0.80      0.87      0.83     17202

avg / total       0.76      0.77      0.76     25905



In [12]:
freq = vectorizer.transform(tweets.text)
predict_tweets = modelo.predict(freq)

In [13]:
tweets['predicted'] = predict_tweets
tweets.head()

,id,text,sentiment,predicted
0,7895,hoje vídeo abrindo emp,Positivo,Positivo
1,7965,passar tarde estudar olha q bonito,Positivo,Positivo
2,3610,tava gaja torneio ai filha puta ainda tentei a...,Positivo,Positivo
3,1139,chateadissima eliminação red canids diria sent...,Negativo,Negativo
4,7386,tô pensando fazer colinha professora é esperta,Negativo,Negativo


In [14]:
tn, fp, fn, tp = metrics.confusion_matrix(tweets['sentiment'], tweets['predicted']).ravel()
print("Nos dados de teste temos:\n%d verdadeiros negativos\n%d falsos positivos\n%d falsos negativos e \n%d verdadeiros positivos" % (tn, fp, fn, tp))

Nos dados de teste temos:
16971 verdadeiros negativos
231 falsos positivos
1171 falsos negativos e 
7532 verdadeiros positivos


### Resultados do Modelo - Teste

Aqui usaremos tweets relativos ao assassinado da vereadora Marielle Franco, são mais de 1600 tweets, para podermos calcular as mesmas métricas calculadas para o treino, 84 tweets foram manualmente classificados em positivos e negativos.

Os dados de teste passaram pelo mesmo processo de padronização que os de treino.

In [15]:
marielle_tweets = pd.read_csv("marielle_tweets.csv", encoding='utf-8').dropna(subset=['text'])
marielle_tweets.head()

,date,text,sentiment
0,2018-07-30 23:40:15,"RT @joaoassi2: #mariellepresente, não só quand...",1
1,2018-07-30 23:31:31,"RT @joaoassi2: #mariellepresente, não só quand...",1
2,2018-07-30 23:30:52,"#mariellepresente, não só quando convém! https...",1
3,2018-07-30 23:14:22,RT @abdalafarah: Hoje Marielle Franco completa...,1
4,2018-07-30 23:12:12,RT @alinecpiva: 29 congressistas estadunidense...,NaN


In [16]:
marielle_tweets.shape

(1634, 3)

Com a padronização os dados tem a seguinte aparência:

In [17]:
marielle_tweets.text = marielle_tweets.text.apply(standardize_text)
marielle_tweets.sentiment = marielle_tweets.sentiment.apply(sentiment_to_name)

marielle_tweets.head()

,date,text,sentiment
0,2018-07-30 23:40:15,mariellepresente convém,Positivo
1,2018-07-30 23:31:31,mariellepresente convém,Positivo
2,2018-07-30 23:30:52,mariellepresente convém,Positivo
3,2018-07-30 23:14:22,hoje marielle franco completaria anos idade v...,Positivo
4,2018-07-30 23:12:12,congressistas estadunidenses incluindo enviar...,NaN


In [18]:
marielle_tweets_freq = vectorizer.transform(marielle_tweets.text)
marielle_predict = modelo.predict(marielle_tweets_freq)

In [19]:
marielle_tweets['predicted'] = marielle_predict

Em função de alguns erros de formatação nos dados de treino, um tratamento especial deverá ser aplicado para remover as observações mal formatadas

In [20]:
marielle_t_class = marielle_tweets.copy()
marielle_t_class= marielle_t_class[(marielle_t_class.sentiment == "Positivo") | (marielle_t_class.sentiment == "Negativo")]
marielle_t_class.dropna(inplace=True)

In [127]:
marielle_t_class.head()

,date,text,sentiment,predicted
0,2018-07-30 23:40:15,mariellepresente convém,Positivo,Positivo
1,2018-07-30 23:31:31,mariellepresente convém,Positivo,Positivo
2,2018-07-30 23:30:52,mariellepresente convém,Positivo,Positivo
3,2018-07-30 23:14:22,hoje marielle franco completaria anos idade vi...,Positivo,Negativo
5,2018-07-30 23:12:04,dias passaram matou marielle mandou matar mari...,Positivo,Negativo


In [21]:
tn, fp, fn, tp = metrics.confusion_matrix(marielle_t_class['sentiment'], marielle_t_class['predicted']).ravel()
print("Nos dados de teste temos:\n%d verdadeiros negativos\n%d falsos positivos\n%d falsos negativos e \n%d verdadeiros positivos" % (tn, fp, fn, tp))

Nos dados de teste temos:
13 verdadeiros negativos
12 falsos positivos
48 falsos negativos e 
11 verdadeiros positivos


In [22]:
marielle_class_freq = vectorizer.transform(marielle_t_class.text)

In [23]:
resultados = cross_val_predict(modelo, marielle_class_freq, marielle_t_class.sentiment, cv = 10)
print("A acurácia do modelo é %.2f nos dados de teste." % metrics.accuracy_score(marielle_t_class.sentiment, resultados))

A acurácia do modelo é 0.85 nos dados de teste.


In [24]:
print("O quadro abaixo mostra os valores de precision, recall e f1-score para os dados de teste\n")
print(metrics.classification_report(marielle_t_class.sentiment, resultados, sentimentos))

O quadro abaixo mostra os valores de precision, recall e f1-score para os dados de teste

             precision    recall  f1-score   support

   Positivo       0.82      1.00      0.90        59
   Negativo       1.00      0.48      0.65        25

avg / total       0.87      0.85      0.83        84



Percebemos que o modelo apresenta valores elevados para todas as métricas nas quais foi submetido, tanto no treino quanto no teste.

Os tweets com a classe inferida pelo modelo podem ser baixados com o seguinte comando:

In [26]:
marielle_tweets.to_csv("tweets_classificados_marielle.csv")